# Imports, load data and trained target

In [28]:
from mnist import MNIST
import pickle as pkl

# mndata = MNIST('D:/raw/')

# X_train, y_train = mndata.load_training()
# X_test, y_test = mndata.load_testing()



In [35]:
X_train = np.array(X_train).reshape(-1, 28, 28)
X_test = np.array(X_test).reshape(-1, 28, 28)
y_train = np.array(y_train)
y_test = np.array(y_test)

X_train.shape, X_test.shape

idx_test2 = list()
idx_test1 = list()
for c in range(10):
    idx_c = np.argwhere(y_test == c).squeeze()
    idx_c2 = list(np.random.choice(idx_c, replace=False, size=200).astype(int))
    
    idx_c1 = set(list(idx_c)).difference(set(idx_c2))
    print(len(idx_c1))
    
    idx_test2.extend(list(idx_c2))
    idx_test1.extend(list(idx_c1))

X_test2 = X_test[idx_test2]
y_test2 = y_test[idx_test2]

X_test1 = X_test[idx_test1]
y_test1 = y_test[idx_test1]

f = open('X_test2.p', 'wb')
pkl.dump(X_test2, f)
np.savetxt('y_test2.txt', y_test2)

f = open('X_test1.p', 'wb')
pkl.dump(X_test1, f)
np.savetxt('y_test1.txt', y_test1)

f = open('X_train.p', 'wb')
pkl.dump(X_train, f)
np.savetxt('y_train.txt', y_train)

X_test1.shape, X_test2.shape

780
935
832
810
782
692
758
828
774
809


((8000, 28, 28), (2000, 28, 28))

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import sys
import pickle as pkl

import torch
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader

from foolbox.models import PyTorchModel
from foolbox.criteria import Misclassification
from foolbox.attacks import GradientSignAttack
from foolbox.adversarial import Adversarial

from src.network import load_target
from src.utils import progress_bar

from IPython.display import clear_output

def load_train_data(root, batch_size, percent_train=0.8, shuffle=True):
    train_data = datasets.MNIST(root=root, train=True, transform=transforms.ToTensor())
    valid_data = datasets.MNIST(root=root, train=True, transform=transforms.ToTensor())
    num_train = len(train_data)
    
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(percent_train * num_train)
    train_idx, valid_idx = indices[:split], indices[split:]
    
    
    
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    
    train_loader = DataLoader(
        train_data, batch_size=batch_size, sampler=train_sampler
    )
    valid_loader = DataLoader(
        valid_data, batch_size=batch_size, sampler=valid_sampler
    )
    
    return (train_loader, valid_loader)

train_loader, valid_loader = load_train_data('D:', batch_size=30)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
trained_target = load_target(trained=True)
trained_target = PyTorchModel(trained_target, bounds=(0, 255), num_classes=10)

criterion = Misclassification()

attack = GradientSignAttack(
    model=trained_target, criterion=criterion
)

# Generate adversarial examples

In [4]:
print('STARTING')
x_adv = np.zeros((0, 28, 28))

# num_train = len(train_loader.sampler.indices)
# num_batches = int(np.ceil(num_train / train_loader.batch_size))

train_data = datasets.MNIST(
    root='D:/', train=True, transform=transforms.ToTensor()
)

missing = []
for i, (X_train, y_train) in progress_bar(enumerate(train_data), mx=len(train_data)):
#     for i in range(X_train.shape[0]):
    adv = attack(X_train.numpy(), y_train.numpy())
    if adv is None:
        adv = np.ones((1, 28, 28)) * np.nan
    x_adv = np.vstack([x_adv, adv])

np.savetxt('adversarial_examples_mnist_train.txt', x_adv)

STARTING


IntProgress(value=0, max=60000)

/home/cole/.virtualenvs/RL/lib/python3.5/site-packages/foolbox/attacks/base.py:129: UserWarning: Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
  warnings.warn('Not running the attack because the original input'


KeyboardInterrupt: 

In [ ]:

# X_ADV = np.zeros((0, 28, 28))
# # for x in x_adv:
# #     X_ADV = np.vstack([X_ADV, x.reshape(1, 28, 28)])
    
# print('FINISHED')



In [ ]:
i = np.random.randint(len(x_adv))

y_pred = trained_target._model(
    torch.Tensor(x_adv[i]).view(1, 28, 28)
).detach().numpy().argmax()

f, ax = plt.subplots(figsize=(12, 7))
ax.imshow(x_adv[i].reshape(28, 28), cmap=cm.binary)
ax.set_title('Predicted: %d' % y_pred)
plt.show()
